# Import Libraries

In [1]:
import os
import cv2
import requests
import zipfile
import numpy as np
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
from tensorflow.keras import layers, models

In [ ]:
CKAN_DATASET_ID = "b987cf48-1b6c-44c7-981a-d25da07f1cd3"
DATASET_DIRECTORY = os.getcwd() + f"\\{CKAN_DATASET_ID}\\Brain MRI"
TEST_SIZE = 0.2
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']


# Get Dataset From CKAN With URL

In [3]:
# Extracting zip file
def extract_zip(ckan_dataset_id: str):
    with zipfile.ZipFile(f'{ckan_dataset_id}.zip', 'r') as zip_ref:
        zip_ref.extractall(f'{ckan_dataset_id}')

#  {{CKAN_API_URL}} = http://192.168.1.15:30020 
#  b987cf48-1b6c-44c7-981a-d25da07f1cd3
# Loading dataset
def load_data(ckan_dataset_id: str):
    url = "{{CKAN_API_URL}}" + f"/api/3/action/package_show?id={ckan_dataset_id}"
    response = requests.get(url, verify=False)
    fetch_data = response.json()
    resources = fetch_data["result"]["resources"]
    zip_resource = None
    for resource in resources:
        if resource['format'].lower() == 'zip':
            zip_resource = resource
            break
    if zip_resource is None:
        raise ValueError("No zip file found in the dataset")

    zip_url = zip_resource['url']
    zip_response = requests.get(zip_url)
    with open(f'{ckan_dataset_id}.zip', 'wb') as f:
        f.write(zip_response.content)

    extract_zip(ckan_dataset_id)
    os.remove(f'{ckan_dataset_id}.zip')

In [ ]:
load_data(CKAN_DATASET_ID)

# Read Data

In [ ]:
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join(DATASET_DIRECTORY,i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

# Split Dataset


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=TEST_SIZE,random_state=101)

In [12]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

# Train Model


In [17]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='sigmoid')
])


In [18]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate Model Performance

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


In [ ]:
# Generate predictions
y_pred = model.predict(X_test)
# Convert predictions to binary classes
y_pred_classes = np.round(y_pred)

# Since y_test and y_pred_classes may have different shapes, we need to reshape them
y_test_reshaped = y_test.reshape(-1)
y_pred_classes_reshaped = y_pred_classes.reshape(-1)

print(classification_report(y_test_reshaped, y_pred_classes_reshaped))


# Save Model

In [28]:
import cloudpickle
# Save the trained model using cloudpickle
with open('classification_model.pkl', 'wb') as f:
    cloudpickle.dump(y_pred, f)